IMPORTACION DE LIBRERIAS

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import os
%matplotlib inline

RUTA DE LAS CARPETAS

In [ ]:
train_data_dir = 'dataset/train'
test_data_dir = 'dataset/test'

TAMAÑO DE LAS IMAGENES DE ENTRADA Y NUMERO DE CLASES

In [ ]:
input_shape = (224, 224, 3)
num_classes = 8

CONFIGURACIÓN DEL GENERADOR DE IMAGENES

In [ ]:
train_datagen = image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

GENERAR LOS DATOS DE ENTRENAMIENTO

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

CREACION DEL MODELO BASE MOBILENETV2

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

ADICION DE CAPAS PERSONALIZADAS

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

COMBINACION DEL MODELO CON LAS NUEVAS CAPAS

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

CONGELADO DE LAS CAPAS DEL MODELO BASE

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

COMPILACION DEL MODELO

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

CONFIGURACION DEL ENTRENAMIENTO

In [ ]:
batch_size = 32
epochs = 50

ENTRENAMIENTO

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs
)

EVALUACION DEL MODELO CON EL CONJUNTO DE PRUEBA

In [ ]:
test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_accuracy = model.evaluate(test_generator)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

PREDICCION DEL CONJUNTO DE PRUEBAS

In [ ]:
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

OBTENCION DE LAS ETIQUETAS DE LAS CLASES

In [ ]:
class_labels = list(test_generator.class_indices.keys())

IMPRIMIR LA IMAGEN Y LA CLASE PREDICHA

In [ ]:
for i in range(len(test_generator.filenames)):
    for i in range(len(test_generator.filenames)):
    filename = test_generator.filenames[i]
    true_label = filename.split('/')[0]  # Extraer la etiqueta verdadera de la ruta de la imagen
    predicted_label = class_labels[y_pred_classes[i]]

    # Cargar y mostrar la imagen
    img = cv2.imread(os.path.join(test_data_dir, filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    print("Imagen:", filename)
    print("Etiqueta verdadera:", true_label)
    print("Clase predicha:", predicted_label)
    print("---------------------")

MATRIZ DE CONFUSION

In [ ]:
cm = confusion_matrix(test_generator.classes, y_pred_classes)

In [ ]:
fig, ax = plt.subplots()

In [ ]:
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)

In [ ]:
ax.set_xlabel('Predicción')
ax.set_ylabel('Etiqueta verdadera')
ax.set_xticklabels(class_labels, rotation=45)
ax.set_yticklabels(class_labels, rotation=0)

VISUALIZACION DE LA MATRIZ

In [ ]:
plt.title('Matriz de Confusión')

In [ ]:
plt.show()

In [ ]:
model.save('models/modelo.h5')